# Backtesting ML Classification-Based

In [ ]:
???

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],45536.090993
Equity Peak [$],59377.806997
Return [%],355.36091


## Load the model

In [6]:
import pickle

In [7]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [8]:
model_dt

DecisionTreeClassifier(max_depth=15)

## Load the data

In [9]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP


## Backtesting.py Library

### Create your Strategy Class

In [11]:
from backtesting import Backtest, Strategy

In [ ]:
Backtest(???)

In [14]:
df_explanatory = df.drop(columns=['change_tomorrow', 'change_tomorrow_direction'])

In [15]:
model_dt.predict(X=df_explanatory)

array(['UP', 'UP', 'UP', ..., 'UP', 'DOWN', 'UP'], dtype=object)

In [19]:
explanatory_today = df_explanatory.iloc[[-1],:]

In [22]:
forecast_tomorrow = model_dt.predict(explanatory_today)[0]

In [40]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt

    def next(self):
        explanatory_today = df_explanatory.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy

#### Calculate Purchase Recommendation

##### Buy if it goes up

In [38]:
list_buy_sell = []

In [39]:
for tomorrow_direction in df.change_tomorrow_direction:
  if tomorrow_direction == 'UP':
    list_buy_sell.append(1)
  elif tomorrow_direction == 'DOWN':
    list_buy_sell.append(-1)

In [42]:
df['buy_sell'] = list_buy_sell

In [43]:
df[['change_tomorrow_direction', 'buy_sell']].head(10)

,change_tomorrow_direction,buy_sell
Date,,
2016-12-08,UP,1
2016-12-09,UP,1
2016-12-12,UP,1
2016-12-13,DOWN,-1
2016-12-14,DOWN,-1
2016-12-15,DOWN,-1
2016-12-16,UP,1
2016-12-19,DOWN,-1
2016-12-20,DOWN,-1


##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [48]:
list_buy_sell = []
already_bought = False

In [49]:
for tomorrow_direction in df.change_tomorrow_direction:
  if tomorrow_direction == 'UP' and already_bought == False:
    list_buy_sell.append(1)
    already_bought = True
  elif tomorrow_direction == 'DOWN' and already_bought == True:
    list_buy_sell.append(-1)
    already_bought = False
  else:
    list_buy_sell.append(0)

In [50]:
df['buy_sell_track'] = list_buy_sell

In [51]:
df[['change_tomorrow_direction', 'buy_sell', 'buy_sell_track']].head(10)

,change_tomorrow_direction,buy_sell,buy_sell_track
Date,,,
2016-12-08,UP,1,1
2016-12-09,UP,1,0
2016-12-12,UP,1,0
2016-12-13,DOWN,-1,-1
2016-12-14,DOWN,-1,0
2016-12-15,DOWN,-1,0
2016-12-16,UP,1,1
2016-12-19,DOWN,-1,-1
2016-12-20,DOWN,-1,0


#### Add conditions to the strategy

In [52]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = df_explanatory.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [54]:
bt = Backtest(data = df_explanatory, strategy = ClassificationUP,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [58]:
results = bt.run()

### Interpret backtesting results

In [59]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],45536.090993
Equity Peak [$],59377.806997
Return [%],355.36091


Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)

Classification Model | Regression Model
-|-
![](src/pred_classification.png) | ![](src/pred_regression.png)